In [672]:
# ===== Cell 1: Imports, paths, fonts =====
import pandas as pd
import numpy as np
from pathlib import Path
import json
import matplotlib.pyplot as plt
import ast

# ----- 폰트(한글) 설정 -----
plt.rcParams['axes.unicode_minus'] = False
# macOS
plt.rcParams['font.family'] = 'AppleGothic'

# ----- 파일 경로 (필요시 수정) -----
POP_FILES = [
    "outputs/241103_park_popweighted_distance_admin_dong_allradii.csv",
    "outputs/241106_park_popweighted_distance_admin_dong_allradii.csv",
    "outputs/250810_park_popweighted_distance_admin_dong_allradii.csv",
    "outputs/250812_park_popweighted_distance_admin_dong_allradii.csv",
]

WALK_NON_JSON = "walking_time_data/parks_accessibility_nonhangang.json"
WALK_HAN_JSON  = "walking_time_data/parks_accessibility_hangang.json"

PARK_NON_JSON = "parking_data/parking_nonhangang_final.json"
PARK_HAN_JSON = "parking_data/parking_hangang.json"

OUT_DIR = Path("./results")
OUT_DIR.mkdir(parents=True, exist_ok=True)

RADIUS_LIST = [500, 1000, 2000]


In [673]:
# ===== Weight Settings (가중치 설정) =====

# 항목별 가중치 (도보, 주차, 인구)
WALK_W = 0.6   # 도보 접근성 중요도
PARK_W = 0.2   # 주차 접근성 중요도
POP_W  = 0.2   # 인구밀집 지역 접근성 중요도

# 반경별 가중치 (500m, 1000m, 2000m)
radii_weights = {
    500: 0.6,   # 500m 반경
    1000: 0.3,  # 1000m 반경
    2000: 0.1   # 2000m 반경
}

In [674]:
# ===== Cell 2: Population-distance (multi-date) → aggregate → wide + scoring =====

def load_pop_files(files):
    dfs = []
    for f in files:
        df = pd.read_csv(f, encoding="utf-8")
        # 컬럼 존재 확인(필수)
        needed = ["날짜","공원명","반경(m)","인구가중_평균거리(m)"]
        missing = [c for c in needed if c not in df.columns]
        if missing:
            raise ValueError(f"{f}에 필요한 컬럼이 없습니다: {missing}")
        # 거리 0 → NaN (집계구 없음 등)
        df["인구가중_평균거리(m)"] = pd.to_numeric(df["인구가중_평균거리(m)"], errors="coerce")
        df.loc[df["인구가중_평균거리(m)"] == 0, "인구가중_평균거리(m)"] = np.nan
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

pop_long = load_pop_files(POP_FILES)

# 날짜 종합(평균). 공원명+반경별 평균거리
pop_agg = (pop_long
           .groupby(["공원명", "반경(m)"], as_index=False)["인구가중_평균거리(m)"]
           .mean())

# wide 피벗: 평균거리_500m / 1000m / 2000m
pop_wide = pop_agg.pivot(index="공원명", columns="반경(m)", values="인구가중_평균거리(m)").reset_index()
pop_wide.columns = ["공원명"] + [f"평균거리_{c}m" for c in pop_wide.columns[1:]]

# 정규화 함수(작을수록 좋음 → reverse=True)
def minmax_score(series, reverse=False):
    s = pd.to_numeric(series, errors="coerce")
    s = s.replace(0, np.nan)
    mn, mx = s.min(), s.max()
    if pd.isna(mn) or pd.isna(mx) or mn == mx:
        # 모두 같은 값이거나 결측이면 50점 고정
        return pd.Series([50]*len(s), index=s.index)
    norm = (s - mn) / (mx - mn)
    if reverse:
        norm = 1 - norm
    return (norm * 100).round(3)

# 반경별 인구점수 생성
for r in RADIUS_LIST:
    col = f"평균거리_{r}m"
    if col in pop_wide.columns:
        pop_wide[f"인구점수_{r}m"] = minmax_score(pop_wide[col], reverse=True)

# 확인
pop_wide.head()


,공원명,평균거리_500m,평균거리_1000m,평균거리_2000m,인구점수_500m,인구점수_1000m,인구점수_2000m
0,강서한강공원,NaN,NaN,NaN,NaN,NaN,NaN
1,경의선숲길,NaN,770.950773,1322.309164,NaN,21.481,40.451
2,고척근린공원,NaN,NaN,1422.827460,NaN,NaN,32.254
3,관악산공원(관악산),NaN,NaN,NaN,NaN,NaN,NaN
4,광나루한강공원,NaN,NaN,1322.489838,NaN,NaN,40.437


In [675]:
# ===== Cell 3: Walking JSONs → tidy → final_walk_min → score =====

def extract_final_walk_min(row):
    # '최종대표시간' 우선 사용
    v = row.get("최종대표시간", None)
    if isinstance(v, (int, float)) and not pd.isna(v):
        return float(v)
    # 없으면 지하철/버스의 대표시간 평균 시도
    def get_rep(d, key_list=("대표시간","median","중앙값","평균")):
        if isinstance(d, dict):
            for k in key_list:
                if k in d and isinstance(d[k], (int, float)):
                    return float(d[k])
    t_sub = get_rep(row.get("지하철", {}))
    t_bus = get_rep(row.get("버스", {}))
    vals = [x for x in [t_sub, t_bus] if isinstance(x, (int, float))]
    return float(np.mean(vals)) if vals else np.nan

def walk_json_to_df(path, is_hangang=False):
    jd = pd.read_json(path)
    df = jd.T.reset_index().rename(columns={"index":"공원명"})
    # 중복 컬럼 제거
    df = df.loc[:, ~df.columns.duplicated()]
    # 대표시간 뽑기
    df["final_walk_min"] = df.apply(extract_final_walk_min, axis=1)
    # 구 컬럼 존재할 수 있음
    gu_col = "구" if "구" in df.columns else None
    out = df[["공원명","final_walk_min"] + ([gu_col] if gu_col else [])].copy()
    out["구분"] = "한강" if is_hangang else "비한강"
    return out

walk_non_df = walk_json_to_df(WALK_NON_JSON, is_hangang=False)
walk_han_df = walk_json_to_df(WALK_HAN_JSON,  is_hangang=True)

# 도보점수(낮을수록 ↑)
def to_walk_score(df):
    s = pd.to_numeric(df["final_walk_min"], errors="coerce")
    score = minmax_score(s, reverse=True)
    return score

walk_non_df["도보점수"] = to_walk_score(walk_non_df)
walk_han_df["도보점수"] = to_walk_score(walk_han_df)

walk_non_df.head()


,공원명,final_walk_min,구,구분,도보점수
0,도산근린공원,10.25,강남구,비한강,76.821
1,올림픽공원,19.00,송파구,비한강,53.642
2,송파나루근린공원(석촌호수),7.75,송파구,비한강,83.444
3,인왕산도시자연공원(인왕산),22.00,종로구,비한강,45.695
4,낙산공원,15.50,종로구,비한강,62.914


In [676]:
# ===== Cell 4 (fixed): Parking JSONs → tidy → parking_score(0~100) =====

def as_bool(x):
    """다양한 표현(True/False/'true'/'false'/'Y'/'N' 등)을 안전하게 bool로 변환"""
    if isinstance(x, bool):
        return x
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return False
    s = str(x).strip().lower()
    return s in {"true","1","y","yes","t","가능","있음","enable","enabled"}

def safe_float(x, default=np.nan):
    try:
        if isinstance(x, str) and x.strip()=="":
            return default
        return float(x)
    except:
        return default

def join_notes(val):
    """notes가 list/str/None 어느 형태든 문자열로 정규화"""
    if isinstance(val, list):
        return " ".join(map(str, val))
    if val is None or (isinstance(val, float) and pd.isna(val)):
        return ""
    return str(val)

def parking_score_row(row):
    # 필드 안전 파싱
    has = row.get("has_parking", None)
    lots = safe_float(row.get("lots_count", np.nan))
    spaces = safe_float(row.get("spaces", np.nan))
    accessible_only = as_bool(row.get("accessible_only", False))
    external = as_bool(row.get("external_parking", False))
    notes = join_notes(row.get("notes", ""))

    # base
    if has is False or (isinstance(has, str) and has.strip().lower() in {"false","0","n","no","없음"}):
        base = 0.0
        if external:  # 외부 주차장 언급 시 소폭 가산
            base = 20.0
    elif has is True or (isinstance(has, str) and has.strip().lower() in {"true","1","y","yes","있음"}):
        base = 20.0
    else:
        base = 10.0  # 정보 불확실

    # lots (최대 5개 반영, 0~40점)
    lots_val = 0.0
    if pd.notna(lots):
        lots_val = max(0.0, min(lots, 5.0)) * 8.0

    # spaces (최대 1000면 반영, 0~40점)
    spaces_val = 0.0
    if pd.notna(spaces):
        s = max(0.0, min(spaces, 1000.0))
        spaces_val = (s / 1000.0) * 40.0

    score = base + lots_val + spaces_val  # 0~100 기본 캡
    if accessible_only:
        score = min(score, 40.0)          # 장애인전용만 가능 시 상한 40점

    # 불확실/추정/약/이상 등의 키워드가 있으면 -10%
    if any(k in notes for k in ["약", "이상", "잘모르겠", "추정", "approximately", "about"]):
        score *= 0.9

    return float(np.clip(score, 0, 100))

def parking_json_to_df(path, is_hangang=False):
    jd = pd.read_json(path)
    df = jd.T.reset_index().rename(columns={"index":"공원명"})
    df = df.loc[:, ~df.columns.duplicated()]

    # 점수 계산
    df["parking_score"] = df.apply(parking_score_row, axis=1)

    # 구 컬럼 있을 수도 있음
    gu_col = "구" if "구" in df.columns else None
    out_cols = ["공원명","parking_score"] + ([gu_col] if gu_col else [])
    out = df[out_cols].copy()
    out["구분"] = "한강" if is_hangang else "비한강"
    return out

# 실제 변환 실행
park_non_df = parking_json_to_df(PARK_NON_JSON, is_hangang=False)
park_han_df  = parking_json_to_df(PARK_HAN_JSON,  is_hangang=True)

park_non_df.head()


,공원명,parking_score,구,구분
0,도산근린공원,0.00,강남구,비한강
1,율현공원,0.00,강남구,비한강
2,경의선숲길,0.00,마포구,비한강
3,문화비축기지,24.28,마포구,비한강
4,올림픽공원,100.00,송파구,비한강


In [677]:
# ===== Cell 5 (replaced): Merge per park (Non-Hangang / Hangang), compute composite & ranks =====
import ast

# 0) 한강 공원 명시 목록 (네 목록으로 교체 가능)
HAN_LIST = [
    "뚝섬한강공원","잠원한강공원","여의도한강공원","반포한강공원","망원한강공원",
    "잠실한강공원","양화한강공원","난지한강공원","이촌한강공원","광나루한강공원","강서한강공원"
]

# 이름 정규화(공백/대소문자 차이 방지)
def _norm(s):
    if pd.isna(s):
        return ""
    return str(s).strip().lower()

HAN_SET = { _norm(x) for x in HAN_LIST } \
          | { _norm(x) for x in walk_han_df["공원명"].unique() } \
          | { _norm(x) for x in park_han_df["공원명"].unique() }

def is_hangang_name(name: str) -> bool:
    ns = _norm(name)
    # 1) 명시 목록/한강 JSON 기반 세트에 포함
    if ns in HAN_SET:
        return True
    # 2) 이름에 '한강공원' 포함
    return "한강공원" in ns

# 1) pop_wide를 한강/비한강으로 분리
pop_wide_han = pop_wide[ pop_wide["공원명"].apply(is_hangang_name) ].copy()
pop_wide_non = pop_wide[~pop_wide["공원명"].apply(is_hangang_name)].copy()

# 2) 병합 함수 (구 정보는 walk 우선, 없으면 parking 보충)
def merge_and_choose_gu(pop_wide_sub, walk_df, park_df):
    m = (pop_wide_sub
         .merge(
             walk_df[["공원명","final_walk_min","도보점수"] + (["구"] if "구" in walk_df.columns else [])],
             on="공원명", how="left", suffixes=("","_walk")
         )
         .merge(
             park_df[["공원명","parking_score"] + (["구"] if "구" in park_df.columns else [])],
             on="공원명", how="left", suffixes=("","_park")
         ))

    def parse_gu(val):
        if isinstance(val, list):
            return val
        if pd.isna(val):
            return None
        s = str(val).strip()
        if s.startswith("[") and s.endswith("]"):
            try:
                lst = ast.literal_eval(s)
                if isinstance(lst, list):
                    return lst
            except:
                pass
        return s

    # 구 컬럼 통합
    if "구" in m.columns:
        m["구"] = m["구"].apply(parse_gu)
    else:
        m["구"] = None
    if "구_walk" in m.columns:
        w = m["구_walk"].apply(parse_gu)
        m["구"] = m["구"].where(m["구"].notna(), w)
    if "구_park" in m.columns:
        p = m["구_park"].apply(parse_gu)
        m["구"] = m["구"].where(m["구"].notna(), p)

    m = m.drop(columns=[c for c in ["구_walk","구_park"] if c in m.columns], errors="ignore")
    m = m.loc[:, ~m.columns.duplicated(keep="first")]
    return m

# 3) 분리된 pop_wide로 각각 병합
non_df = merge_and_choose_gu(pop_wide_non, walk_non_df, park_non_df).copy()
han_df  = merge_and_choose_gu(pop_wide_han, walk_han_df,  park_han_df).copy()

# 4) 반경별 가중 합산 점수 (결측은 각 DF 중앙값 → 중앙값도 NaN이면 0 대체)
def add_composite_scores(df, radii=RADIUS_LIST, walk_w=WALK_W, park_w=PARK_W, pop_w=POP_W):
    for r in radii:
        pop_series  = df.get(f"인구점수_{r}m", pd.Series(np.nan, index=df.index))
        walk_series = df.get("도보점수", pd.Series(np.nan, index=df.index))
        park_series = df.get("parking_score", pd.Series(np.nan, index=df.index))

        pop_fill  = 0 if pd.isna(pop_series.median(skipna=True))  else pop_series.median(skipna=True)
        walk_fill = 0 if pd.isna(walk_series.median(skipna=True)) else walk_series.median(skipna=True)
        park_fill = 0 if pd.isna(park_series.median(skipna=True)) else park_series.median(skipna=True)

        df[f"총점_{r}m"] = (
            pop_series.fillna(pop_fill)  * pop_w +
            walk_series.fillna(walk_fill)* walk_w +
            park_series.fillna(park_fill)* park_w
        )
    return df

non_df = add_composite_scores(non_df)
han_df  = add_composite_scores(han_df)

# 5) 공원별 순위 생성(반경별) — NaN 순위는 비워두기(nullable Int64)
def rank_per_radius(df, radii=RADIUS_LIST):
    out = df.copy()
    for r in radii:
        ranks = out[f"총점_{r}m"].rank(ascending=False, method="min")
        out[f"rank_{r}m"] = ranks.astype("Int64")   # NaN 허용
    return out

non_rank = rank_per_radius(non_df)
han_rank  = rank_per_radius(han_df)

# 6) 저장(공원별 순위) — 비한강과 한강을 완전히 분리 저장
non_rank_out_csv = OUT_DIR / "parks_rank_nonhangang.csv"
non_rank_out_json = OUT_DIR / "parks_rank_nonhangang.json"
han_rank_out_csv  = OUT_DIR / "parks_rank_hangang.csv"
han_rank_out_json = OUT_DIR / "parks_rank_hangang.json"

non_rank.to_csv(non_rank_out_csv, index=False, encoding="utf-8-sig")
non_rank.to_json(non_rank_out_json, orient="records", force_ascii=False, indent=2)
han_rank.to_csv(han_rank_out_csv, index=False, encoding="utf-8-sig")
han_rank.to_json(han_rank_out_json, orient="records", force_ascii=False, indent=2)

print("저장 완료(분리):")
print(non_rank_out_csv, non_rank_out_json)
print(han_rank_out_csv,  han_rank_out_json)
display(non_rank.head())
display(han_rank.head())


저장 완료(분리):
results/parks_rank_nonhangang.csv results/parks_rank_nonhangang.json
results/parks_rank_hangang.csv results/parks_rank_hangang.json


,공원명,평균거리_500m,평균거리_1000m,평균거리_2000m,인구점수_500m,인구점수_1000m,인구점수_2000m,final_walk_min,도보점수,구,parking_score,총점_500m,총점_1000m,총점_2000m,rank_500m,rank_1000m,rank_2000m
0,경의선숲길,NaN,770.950773,1322.309164,NaN,21.481,40.451,4.00,93.377,마포구,0.00,65.2399,60.3224,64.1164,14,15,12
1,고척근린공원,NaN,NaN,1422.827460,NaN,NaN,32.254,15.25,63.576,구로구,35.96,54.5513,50.0171,51.7884,32,31,35
2,관악산공원(관악산),NaN,NaN,NaN,NaN,NaN,NaN,10.00,77.483,관악구,29.96,61.6955,57.1613,59.9862,20,21,20
3,국립서울현충원,NaN,NaN,1683.716274,NaN,NaN,10.977,6.50,86.755,동작구,72.00,75.6667,71.1325,68.6484,3,6,7
4,금천체육공원,NaN,NaN,1574.587478,NaN,NaN,19.877,18.00,56.291,금천구,29.04,48.7963,44.2621,43.5580,37,40,42


,공원명,평균거리_500m,평균거리_1000m,평균거리_2000m,인구점수_500m,인구점수_1000m,인구점수_2000m,final_walk_min,도보점수,구,parking_score,총점_500m,총점_1000m,총점_2000m,rank_500m,rank_1000m,rank_2000m
0,강서한강공원,NaN,NaN,NaN,NaN,NaN,NaN,24.50,0.000,None,30.08,6.0160,11.5771,12.2168,11,11,11
1,광나루한강공원,NaN,NaN,1322.489838,NaN,NaN,40.437,14.75,53.425,None,67.56,45.5670,51.1281,53.6544,9,10,9
2,난지한강공원,NaN,586.113552,864.468491,NaN,43.009,77.791,15.50,49.315,None,76.72,44.9330,53.5348,60.4912,10,8,6
3,뚝섬한강공원,NaN,NaN,1582.307110,NaN,NaN,19.247,12.00,68.493,None,78.96,56.8878,62.4489,60.7372,3,3,5
4,망원한강공원,NaN,NaN,1599.220647,NaN,NaN,17.868,13.50,60.274,None,49.28,46.0204,51.5815,49.5940,8,9,10


In [678]:
# ===== Cell 5.5: Fix duplicate '구' columns and ensure a single '구' column =====

import ast

def _parse_gu(val):
    """여러 형태(리스트/문자열/NaN)를 통일해서 리스트로 반환"""
    if val is None or (isinstance(val, float) and pd.isna(val)):
        return None
    if isinstance(val, list):
        return val
    s = str(val).strip()
    if not s:
        return None
    # 문자열로 저장된 리스트 처리
    if s.startswith("[") and s.endswith("]"):
        try:
            v = ast.literal_eval(s)
            if isinstance(v, list):
                return v
        except:
            pass
    return s

def _choose_single_gu_column(df):
    """merge 과정에서 생길 수 있는 '구', '구_walk', '구_park' 등을 하나의 '구'로 통합"""
    # 1) 컬럼 중복 제거(이름이 완전히 같은 컬럼이 중복될 때)
    df = df.loc[:, ~df.columns.duplicated(keep="first")].copy()

    # 2) 최종 '구' 만들기: 구_walk 우선, 없으면 구_park, 그래도 없으면 기존 '구'
    cand_cols = []
    if "구_walk" in df.columns: cand_cols.append("구_walk")
    if "구_park" in df.columns: cand_cols.append("구_park")
    if "구" in df.columns:      cand_cols.append("구")

    if not cand_cols:
        # '구' 정보가 전혀 없으면 빈 리스트 할당
        df["구"] = None
    else:
        # 우선순위대로 선택
        gu_series = None
        for c in cand_cols:
            s = df[c].apply(_parse_gu)
            if gu_series is None:
                gu_series = s
            else:
                # 앞에서 비어있던 곳을 뒤 컬럼 값으로 보충
                mask = gu_series.isna()
                gu_series = gu_series.where(~mask, s)
        df["구"] = gu_series

    # 보조 컬럼 제거
    drop_cols = [c for c in ["구_walk","구_park"] if c in df.columns]
    if drop_cols:
        df = df.drop(columns=drop_cols)

    # 혹시라도 동일 이름 '구'가 중복 생겼으면 다시 한 번 정리
    df = df.loc[:, ~df.columns.duplicated(keep="first")]

    return df

# 비한강/한강 모두 정리
non_rank = _choose_single_gu_column(non_rank)
han_rank  = _choose_single_gu_column(han_rank)

# '구'가 DataFrame이 되는 상황 방지: 혹시라도 같은 이름 중복으로 DataFrame이 됐다면 첫 컬럼만 취함
if isinstance(non_rank["구"], pd.DataFrame):
    non_rank["구"] = non_rank["구"].iloc[:, 0]
if isinstance(han_rank["구"], pd.DataFrame):
    han_rank["구"] = han_rank["구"].iloc[:, 0]


In [679]:
# ===== Cell 6 (fixed-final): District aggregation (Non-Hangang): average & top =====
import ast
import pandas as pd
import numpy as np

def normalize_gu_value(g):
    """단일/리스트/문자열/None → 리스트[str] ('구' 접미사 보정)"""
    if g is None or (isinstance(g, float) and pd.isna(g)):
        return []
    if isinstance(g, list):
        out = []
        for x in g:
            if x is None or (isinstance(x, float) and pd.isna(x)):
                continue
            s = str(x).strip()
            if not s:
                continue
            if not s.endswith("구"):
                s += "구"
            out.append(s)
        return out
    s = str(g).strip()
    if not s:
        return []
    # 문자열로 저장된 리스트 처리
    if s.startswith("[") and s.endswith("]"):
        try:
            lst = ast.literal_eval(s)
            return normalize_gu_value(lst)
        except:
            pass
    if not s.endswith("구"):
        s += "구"
    return [s]

# 0) 혹시 동일 이름 중복 컬럼이 또 있다면 제거
non_rank = non_rank.loc[:, ~non_rank.columns.duplicated(keep="first")].copy()

# 1) '구'가 DataFrame로 남아있는 상황 방지: 첫 컬럼만 채택
if "구" in non_rank.columns and isinstance(non_rank["구"], pd.DataFrame):
    non_rank["구"] = non_rank["구"].iloc[:, 0]

# 2) 구 리스트화
non_exp = non_rank.copy()
non_exp["구_list"] = non_exp["구"].apply(normalize_gu_value)

# 3) explode 전에 기존 '구' 컬럼을 제거해 이름 충돌 방지
non_exp = non_exp.drop(columns=[c for c in non_exp.columns if c == "구"], errors="ignore")

# 4) explode 수행 (리스트 → 행 분해)
non_exp = non_exp.explode("구_list", ignore_index=True)

# 5) explode 결과를 최종 '구'로 승격
non_exp = non_exp.rename(columns={"구_list": "구"})

# 6) 여기서 또 동일 이름 컬럼이 생겼을 수 있으니 한 번 더 정리
non_exp = non_exp.loc[:, ~non_exp.columns.duplicated(keep="first")]

# 7) 문자열 정리 (이 시점의 '구'는 반드시 Series)
non_exp["구"] = non_exp["구"].astype("string").fillna("").str.strip()
non_exp = non_exp.loc[non_exp["구"] != ""].copy()

def aggregate_by_gu(df, radii=[500, 1000, 2000]):
    frames = []
    for r in radii:
        total_col = f"총점_{r}m"
        if total_col not in df.columns:
            continue
        tmp = df[["구", total_col]].dropna().copy()
        if tmp.empty:
            continue
        avg = (tmp.groupby("구", as_index=False)[total_col]
               .mean()
               .rename(columns={total_col: f"구평균_{r}m"}))
        top = (tmp.groupby("구", as_index=False)[total_col]
               .max()
               .rename(columns={total_col: f"구최고_{r}m"}))
        res = avg.merge(top, on="구", how="outer")
        res[f"rank_평균_{r}m"] = res[f"구평균_{r}m"].rank(ascending=False, method="min").astype(int)
        res[f"rank_최고_{r}m"] = res[f"구최고_{r}m"].rank(ascending=False, method="min").astype(int)
        res["반경"] = r
        frames.append(res)
    if frames:
        return pd.concat(frames, ignore_index=True)
    return pd.DataFrame(columns=["구","반경","구평균","구최고","rank_평균","rank_최고"])

gu_scores = aggregate_by_gu(non_exp, radii=[500, 1000, 2000])

# 저장
gu_scores_csv = OUT_DIR / "district_scores_nonhangang.csv"
gu_scores_json = OUT_DIR / "district_scores_nonhangang.json"
gu_scores.to_csv(gu_scores_csv, index=False, encoding="utf-8-sig")
gu_scores.to_json(gu_scores_json, orient="records", force_ascii=False, indent=2)

print("구별 점수 저장 완료:", gu_scores_csv, gu_scores_json)
display(gu_scores.head(20))


구별 점수 저장 완료: results/district_scores_nonhangang.csv results/district_scores_nonhangang.json


,구,구평균_500m,구최고_500m,rank_평균_500m,rank_최고_500m,반경,구평균_1000m,구최고_1000m,rank_평균_1000m,rank_최고_1000m,구평균_2000m,구최고_2000m,rank_평균_2000m,rank_최고_2000m
0,강남구,45.372700,55.3063,23.0,23.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,강동구,52.525000,54.9091,19.0,24.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,강북구,63.186600,66.7393,10.0,12.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,강서구,62.177433,77.4773,14.0,2.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,관악구,65.520700,69.3459,7.0,8.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,광진구,68.051050,97.8820,3.0,1.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,구로구,56.031300,57.5113,17.0,21.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,금천구,48.475300,48.7963,22.0,25.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,노원구,44.957600,67.3867,24.0,10.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,도봉구,68.943100,71.1469,2.0,6.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [680]:
# ===== Cell 7: 공원별(비한강/한강) 점수 테이블 정리 =====
import pandas as pd
import numpy as np
import ast

def _ensure_single_gu_column(df):
    """merge 과정에서 생긴 구 관련 보조 컬럼 정리 → '구' 단일 컬럼으로 통일"""
    df = df.loc[:, ~df.columns.duplicated(keep="first")].copy()
    if "구" not in df.columns:
        cand = [c for c in ["구_walk","구_park"] if c in df.columns]
        if cand:
            df["구"] = df[cand[0]]
    # '구'가 DataFrame이면 첫 컬럼만
    if "구" in df.columns and isinstance(df["구"], pd.DataFrame):
        df["구"] = df["구"].iloc[:, 0]
    return df

def _parse_gu_to_list(g):
    """구 정보를 리스트로 통일"""
    if g is None or (isinstance(g, float) and np.isnan(g)):
        return []
    if isinstance(g, list):
        return [str(x).strip() for x in g if str(x).strip()]
    s = str(g).strip()
    if not s:
        return []
    if s.startswith("[") and s.endswith("]"):
        try:
            lst = ast.literal_eval(s)
            if isinstance(lst, list):
                return [str(x).strip() for x in lst if str(x).strip()]
        except:
            pass
    return [s]

def build_park_table(base_df):
    """non_rank / han_rank에서 공원별 컴포넌트 + 반경별 총점 테이블 생성"""
    df = _ensure_single_gu_column(base_df)

    # '구' 리스트화(다구 관할 지원)
    df["구_list"] = df["구"].apply(_parse_gu_to_list)

    # 필요한 컬럼들만
    keep_cols = ["공원명","도보점수","parking_score","구_list"] + \
                [c for c in df.columns if c.startswith("인구점수_")] + \
                [c for c in df.columns if c.startswith("총점_")]
    keep_cols = [c for c in keep_cols if c in df.columns]
    out = df[keep_cols].drop_duplicates(subset=["공원명"]).reset_index(drop=True)
    return out

# 비한강/한강 공원별 테이블
parks_non = build_park_table(non_rank)
parks_han = build_park_table(han_rank)

# 확인
display(parks_non.head())
display(parks_han.head())


,공원명,도보점수,parking_score,구_list,인구점수_500m,인구점수_1000m,인구점수_2000m,총점_500m,총점_1000m,총점_2000m
0,경의선숲길,93.377,0.00,[마포구],NaN,21.481,40.451,65.2399,60.3224,64.1164
1,고척근린공원,63.576,35.96,[구로구],NaN,NaN,32.254,54.5513,50.0171,51.7884
2,관악산공원(관악산),77.483,29.96,[관악구],NaN,NaN,NaN,61.6955,57.1613,59.9862
3,국립서울현충원,86.755,72.00,[동작구],NaN,NaN,10.977,75.6667,71.1325,68.6484
4,금천체육공원,56.291,29.04,[금천구],NaN,NaN,19.877,48.7963,44.2621,43.5580


,공원명,도보점수,parking_score,구_list,인구점수_500m,인구점수_1000m,인구점수_2000m,총점_500m,총점_1000m,총점_2000m
0,강서한강공원,0.000,30.08,[],NaN,NaN,NaN,6.0160,11.5771,12.2168
1,광나루한강공원,53.425,67.56,[],NaN,NaN,40.437,45.5670,51.1281,53.6544
2,난지한강공원,49.315,76.72,[],NaN,43.009,77.791,44.9330,53.5348,60.4912
3,뚝섬한강공원,68.493,78.96,[],NaN,NaN,19.247,56.8878,62.4489,60.7372
4,망원한강공원,60.274,49.28,[],NaN,NaN,17.868,46.0204,51.5815,49.5940


In [681]:
# ===== Cell 8: 비한강 구별 점수 산출(컴포넌트 평균 + 반경별 종합 평균/최고) =====
import pandas as pd
import numpy as np

def explode_by_gu(df):
    """구 리스트 explode → ('구' 접미사 보정은 여기서는 생략; 입력 데이터에 이미 '구' 형태로 있음)"""
    e = df.explode("구_list", ignore_index=True).rename(columns={"구_list":"구"}).copy()
    e["구"] = e["구"].astype("string").fillna("").str.strip()
    e = e.loc[e["구"] != ""].copy()
    return e

non_exp = explode_by_gu(parks_non)

# 1) 컴포넌트별(도보/주차/인구) 구 평균
comp_cols = [c for c in ["도보점수","parking_score"] if c in non_exp.columns]
# 인구는 반경별로 존재 → 구 평균을 반경별로 모두 만듦
pop_cols = [c for c in non_exp.columns if c.startswith("인구점수_")]

# 구별 컴포넌트 평균(도보/주차)
gu_comp_avg = (non_exp.groupby("구", as_index=False)[comp_cols].mean()) if comp_cols else pd.DataFrame()

# 구별 인구점수 평균(반경별)
if pop_cols:
    gu_pop_avg = non_exp.groupby("구", as_index=False)[pop_cols].mean()
else:
    gu_pop_avg = pd.DataFrame({"구": non_exp["구"].unique()})

# 2) 반경별 종합(총점) → 평균 & 최고
radius_cols = {}
for r in RADIUS_LIST:
    col = f"총점_{r}m"
    if col in non_exp.columns:
        # 평균
        avg = non_exp.groupby("구", as_index=False)[col].mean().rename(columns={col: f"구평균_{r}m"})
        # 최고
        top = non_exp.groupby("구", as_index=False)[col].max().rename(columns={col: f"구최고_{r}m"})
        tmp = avg.merge(top, on="구", how="outer")
        tmp[f"rank_평균_{r}m"] = tmp[f"구평균_{r}m"].rank(ascending=False, method="min").astype(int)
        tmp[f"rank_최고_{r}m"] = tmp[f"구최고_{r}m"].rank(ascending=False, method="min").astype(int)
        radius_cols[r] = tmp

# 3) 하나의 구별 테이블로 합치기
from functools import reduce

tables = []
if not gu_comp_avg.empty:
    tables.append(gu_comp_avg)
if not gu_pop_avg.empty:
    tables.append(gu_pop_avg)
tables += list(radius_cols.values())

if tables:
    gu_all = reduce(lambda L, R: pd.merge(L, R, on="구", how="outer"), tables)
else:
    gu_all = pd.DataFrame(columns=["구"])

# 저장
gu_all_csv = OUT_DIR / "district_scores_components_nonhangang.csv"
gu_all_json = OUT_DIR / "district_scores_components_nonhangang.json"
gu_all.to_csv(gu_all_csv, index=False, encoding="utf-8-sig")
gu_all.to_json(gu_all_json, orient="records", force_ascii=False, indent=2)

print("구별(비한강) 컴포넌트+종합 저장:", gu_all_csv, gu_all_json)
display(gu_all.head(10))


구별(비한강) 컴포넌트+종합 저장: results/district_scores_components_nonhangang.csv results/district_scores_components_nonhangang.json


,구,도보점수,parking_score,인구점수_500m,인구점수_1000m,인구점수_2000m,구평균_500m,구최고_500m,rank_평균_500m,rank_최고_500m,구평균_1000m,구최고_1000m,rank_평균_1000m,rank_최고_1000m,구평균_2000m,구최고_2000m,rank_평균_2000m,rank_최고_2000m
0,강남구,60.265000,0.000000,NaN,14.4780,33.7010,45.372700,55.3063,24,23,39.946550,48.9882,25,24,42.899200,56.4522,24,21
1,강동구,72.185500,0.000000,NaN,24.3835,55.0645,52.525000,54.9091,19,24,48.188000,49.6316,19,23,54.324200,56.6778,16,20
2,강북구,73.509500,49.336000,NaN,NaN,23.1995,63.186600,66.7393,10,12,58.652400,62.2051,11,12,58.612800,60.6584,13,14
3,강서구,75.717333,37.666667,NaN,45.9910,100.0000,62.177433,77.4773,14,2,59.149467,72.9431,10,5,64.633333,75.7680,6,3
4,관악구,72.185000,64.980000,NaN,NaN,NaN,65.520700,69.3459,7,8,60.986500,64.8117,9,9,63.811400,67.6366,9,7
5,광진구,74.172000,45.580000,98.25,37.4185,62.8750,68.051050,97.8820,3,1,61.102900,83.5320,8,1,66.194200,88.7418,2,1
6,구로구,66.556000,34.420000,NaN,NaN,34.4035,56.031300,57.5113,18,21,51.497100,52.9771,18,22,53.698300,55.6082,18,22
7,금천구,60.596000,14.520000,NaN,NaN,29.2450,48.475300,48.7963,23,26,43.941100,44.2621,23,26,45.110600,46.6632,23,26
8,노원구,49.006500,31.700000,NaN,NaN,7.9950,44.957600,67.3867,25,10,40.423400,62.8525,24,10,40.295600,59.7720,25,18
9,도봉구,87.417000,36.396000,NaN,NaN,31.0645,68.943100,71.1469,2,6,64.408900,66.6127,5,8,65.942300,71.2262,3,5


In [682]:
# ===== Cell 9: 비한강 구별 순위+점수 시각화 =====
import matplotlib.pyplot as plt

def plot_gu_bar(df, value_col, title, fname, topn=None):
    t = df[["구", value_col]].dropna().copy()
    if t.empty:
        print(f"[건너뜀] {value_col} 데이터가 없습니다.")
        return None
    t = t.sort_values(value_col, ascending=False)
    if topn:
        t = t.head(topn)
    # 가로 막대
    t = t.sort_values(value_col)  # 아래→위 증가
    plt.figure(figsize=(9, 6))
    plt.barh(t["구"], t[value_col])
    plt.title(title)
    plt.xlabel("점수")
    plt.tight_layout()
    out = OUT_DIR / fname
    plt.savefig(out, dpi=150)
    plt.close()
    return out

# 예: 1000m 기준 평균/최고 각각 시각화
r = 1000
avg_col = f"구평균_{r}m"
top_col = f"구최고_{r}m"

img1 = plot_gu_bar(gu_all, avg_col, f"비한강 구별 평균 종합점수 (반경 {r}m)", f"nonhangang_gu_avg_{r}m.png")
img2 = plot_gu_bar(gu_all, top_col, f"비한강 구별 최고 종합점수 (반경 {r}m)", f"nonhangang_gu_top_{r}m.png")

# 컴포넌트 평균도 시각화(도보/주차/인구_반경별)
if "도보점수" in gu_all.columns:
    img3 = plot_gu_bar(gu_all, "도보점수", "비한강 구별 도보점수 평균", "nonhangang_gu_walk_avg.png")
if "parking_score" in gu_all.columns:
    img4 = plot_gu_bar(gu_all, "parking_score", "비한강 구별 주차점수 평균 ", "nonhangang_gu_parking_avg.png")
for r in RADIUS_LIST:
    pc = f"인구점수_{r}m"
    if pc in gu_all.columns:
        _ = plot_gu_bar(gu_all, pc, f"비한강 구별 인구점수 평균 (반경 {r}m)", f"nonhangang_gu_pop_{r}m.png")

print("저장된 구별 시각화 예시:", img1, img2)


저장된 구별 시각화 예시: results/nonhangang_gu_avg_1000m.png results/nonhangang_gu_top_1000m.png


In [683]:
# ===== Cell 10: 한강공원 종합 순위(공원 기준) + 저장 =====

# 한강 공원: 반경별 총점 평균으로 '한강 종합점수' 생성(원하면 가중 평균으로 바꿔도 됨)
total_cols_han = [c for c in parks_han.columns if c.startswith("총점_")]
if total_cols_han:
    parks_han["한강_종합점수"] = parks_han[total_cols_han].mean(axis=1)
else:
    # 총점이 없다면 컴포넌트 평균으로 대체
    comp_cols_han = [c for c in ["도보점수","parking_score"] + [c for c in parks_han.columns if c.startswith("인구점수_")] if c in parks_han.columns]
    parks_han["한강_종합점수"] = parks_han[comp_cols_han].mean(axis=1) if comp_cols_han else np.nan

# 순위
parks_han["rank"] = parks_han["한강_종합점수"].rank(ascending=False, method="min").astype(int)

# 저장
han_csv = OUT_DIR / "hangang_parks_rank.csv"
han_json = OUT_DIR / "hangang_parks_rank.json"
parks_han.to_csv(han_csv, index=False, encoding="utf-8-sig")
parks_han.to_json(han_json, orient="records", force_ascii=False, indent=2)

# 시각화(Top 11)
t = parks_han[["공원명","한강_종합점수"]].dropna().sort_values("한강_종합점수", ascending=False).head(11)
t = t.sort_values("한강_종합점수")
plt.figure(figsize=(9, 6))
plt.barh(t["공원명"], t["한강_종합점수"])
plt.title("한강 공원 종합 순위 (Top 11)")
plt.xlabel("점수")
plt.tight_layout()
out_img = OUT_DIR / "hangang_parks_rank.png"
plt.savefig(out_img, dpi=150)
plt.close()

print("한강 결과 저장:", han_csv, han_json, out_img)
display(parks_han.sort_values("rank").head(11))


한강 결과 저장: results/hangang_parks_rank.csv results/hangang_parks_rank.json results/hangang_parks_rank.png


,공원명,도보점수,parking_score,구_list,인구점수_500m,인구점수_1000m,인구점수_2000m,총점_500m,총점_1000m,총점_2000m,한강_종합점수,rank
6,양화한강공원,100.000,64.04,[],NaN,NaN,32.704,72.8080,78.3691,79.3488,76.841967,1
7,여의도한강공원,86.301,100.00,[],NaN,NaN,26.318,71.7806,77.3417,77.0442,75.388833,2
9,잠실한강공원,69.863,67.48,[],NaN,23.736,57.034,55.4138,60.1610,66.8206,60.798467,3
3,뚝섬한강공원,68.493,78.96,[],NaN,NaN,19.247,56.8878,62.4489,60.7372,60.024633,4
10,잠원한강공원,63.014,88.88,[],NaN,31.875,24.272,55.5844,61.9594,60.4388,59.327533,5
5,반포한강공원,60.274,92.76,[],NaN,15.399,44.235,54.7164,57.7962,63.5634,58.692000,6
8,이촌한강공원,64.384,66.84,[],NaN,NaN,29.304,51.9984,57.5595,57.8592,55.805700,7
2,난지한강공원,49.315,76.72,[],NaN,43.009,77.791,44.9330,53.5348,60.4912,52.986333,8
1,광나루한강공원,53.425,67.56,[],NaN,NaN,40.437,45.5670,51.1281,53.6544,50.116500,9
4,망원한강공원,60.274,49.28,[],NaN,NaN,17.868,46.0204,51.5815,49.5940,49.065300,10


In [684]:
# ===== Cell 11: 비한강 공원별 최종 테이블 저장 =====

# 반경별 순위 컬럼 붙이기
parks_non_out = parks_non.copy()
for r in RADIUS_LIST:
    col = f"총점_{r}m"
    if col in parks_non_out.columns:
        parks_non_out[f"rank_{r}m"] = parks_non_out[col].rank(ascending=False, method="min").astype(int)

# 컴포넌트 종합(간단 평균)도 하나 추가(원하면 가중치 반영 가능)
comp_cols_non = [c for c in ["도보점수","parking_score"] + [c for c in parks_non_out.columns if c.startswith("인구점수_")] if c in parks_non_out.columns]
if comp_cols_non:
    parks_non_out["컴포넌트_종합(평균)"] = parks_non_out[comp_cols_non].mean(axis=1)

# 저장
non_csv = OUT_DIR / "nonhangang_parks_full_table.csv"
non_json = OUT_DIR / "nonhangang_parks_full_table.json"
parks_non_out.to_csv(non_csv, index=False, encoding="utf-8-sig")
parks_non_out.to_json(non_json, orient="records", force_ascii=False, indent=2)

print("비한강 공원별 최종 저장:", non_csv, non_json)
display(parks_non_out.head(10))


비한강 공원별 최종 저장: results/nonhangang_parks_full_table.csv results/nonhangang_parks_full_table.json


,공원명,도보점수,parking_score,구_list,인구점수_500m,인구점수_1000m,인구점수_2000m,총점_500m,총점_1000m,총점_2000m,rank_500m,rank_1000m,rank_2000m,컴포넌트_종합(평균)
0,경의선숲길,93.377,0.00,[마포구],NaN,21.481,40.451,65.2399,60.3224,64.1164,14,15,12,38.827250
1,고척근린공원,63.576,35.96,[구로구],NaN,NaN,32.254,54.5513,50.0171,51.7884,32,31,35,43.930000
2,관악산공원(관악산),77.483,29.96,[관악구],NaN,NaN,NaN,61.6955,57.1613,59.9862,20,21,20,53.721500
3,국립서울현충원,86.755,72.00,[동작구],NaN,NaN,10.977,75.6667,71.1325,68.6484,3,6,7,56.577333
4,금천체육공원,56.291,29.04,[금천구],NaN,NaN,19.877,48.7963,44.2621,43.5580,37,40,42,35.069333
5,금천폭포근린공원,64.901,0.00,[금천구],NaN,NaN,38.613,48.1543,43.6201,46.6632,39,41,40,34.504667
6,길동생태공원,76.159,0.00,[강동구],NaN,19.681,54.912,54.9091,49.6316,56.6778,31,32,26,37.688000
7,낙산공원,62.914,21.16,[종로구],19.850,29.393,44.973,45.9504,47.8590,50.9750,41,35,36,35.658000
8,남산공원,0.000,40.20,[중구],NaN,6.513,16.292,17.2537,9.3426,11.2984,47,47,47,15.751250
9,달맞이근린공원,72.848,0.00,[성동구],41.341,60.096,35.492,51.9770,55.7280,50.8072,34,24,37,41.955400


In [685]:
# # ===== Cell A: Show & save ALL districts (no top N cut) =====

# def full_district_tables(gu_df, radii=[500, 1000, 2000]):
#     all_tables = {}
#     for r in radii:
#         sub = gu_df.query("반경 == @r").copy()
#         # 보기 좋게 정렬(평균 기준 ↓), 동률은 최고점 기준 ↓
#         sub = sub.sort_values([f"구평균_{r}m", f"구최고_{r}m"], ascending=[False, False])
#         all_tables[r] = sub

#         # 저장(전 구 포함)
#         out_csv = OUT_DIR / f"district_scores_nonhangang_ALL_{r}m.csv"
#         out_json = OUT_DIR / f"district_scores_nonhangang_ALL_{r}m.json"
#         sub.to_csv(out_csv, index=False, encoding="utf-8-sig")
#         sub.to_json(out_json, orient="records", force_ascii=False, indent=2)
#         print(f"[저장] {r}m → {out_csv}, {out_json}")
#     return all_tables

# all_gu_tables = full_district_tables(gu_scores, radii=RADIUS_LIST)

# # 예시: 1000m 반경 전 구 테이블 미리보기
# display(all_gu_tables[1000].head(25))


In [686]:
# ===== Cell B: Plot ALL districts (no topn filtering) =====

def plot_gu_scores_all(gu_df, radius=1000, which="평균", title_prefix="비한강 구별(전체)"):
    col = f"구{which}_{radius}m"
    sub = gu_df.query("반경 == @radius")[["구", col]].dropna().copy()
    # 내림차순 정렬 후, 가로바는 보기 좋게 오름차순으로 다시 정렬
    sub = sub.sort_values(col, ascending=False)
    sub = sub.sort_values(col)  # barh에서 아래→위로 커지게

    plt.figure(figsize=(10, max(6, len(sub) * 0.35)))  # 구가 많으면 자동으로 높이 늘림
    plt.barh(sub["구"], sub[col])
    plt.title(f"{title_prefix} {which} 점수 (반경 {radius}m)")
    plt.xlabel("점수")
    plt.tight_layout()

    out = OUT_DIR / f"plot_gu_all_{which}_{radius}m.png"
    plt.savefig(out, dpi=150)
    plt.close()
    print("그래프 저장:", out)
    return out

# 반경별로 평균/최고 전부 저장
for r in RADIUS_LIST:
    plot_gu_scores_all(gu_scores, radius=r, which="평균")
    plot_gu_scores_all(gu_scores, radius=r, which="최고")


그래프 저장: results/plot_gu_all_평균_500m.png
그래프 저장: results/plot_gu_all_최고_500m.png
그래프 저장: results/plot_gu_all_평균_1000m.png
그래프 저장: results/plot_gu_all_최고_1000m.png
그래프 저장: results/plot_gu_all_평균_2000m.png
그래프 저장: results/plot_gu_all_최고_2000m.png


In [687]:
# ===== Cell C (optional): Wide summary table for all districts =====

def wide_summary_table(gu_df, radii=[500, 1000, 2000]):
    frames = []
    for r in radii:
        sub = gu_df.query("반경 == @r")[["구", f"구평균_{r}m", f"rank_평균_{r}m", f"구최고_{r}m", f"rank_최고_{r}m"]].copy()
        sub = sub.set_index("구")
        sub.columns = [f"{c}_{r}" for c in sub.columns]  # 컬럼에 반경 suffix
        frames.append(sub)
    wide = pd.concat(frames, axis=1).reset_index().rename(columns={"index":"구"})
    # 정렬(1000m 평균 기준 ↓)
    if f"구평균_{radii[1]}m_{radii[1]}" in wide.columns:
        wide = wide.sort_values(f"구평균_{radii[1]}m_{radii[1]}", ascending=False)
    out_csv = OUT_DIR / "district_scores_nonhangang_WIDE_allradii.csv"
    out_json = OUT_DIR / "district_scores_nonhangang_WIDE_allradii.json"
    wide.to_csv(out_csv, index=False, encoding="utf-8-sig")
    wide.to_json(out_json, orient="records", force_ascii=False, indent=2)
    print("요약 Wide 저장:", out_csv, out_json)
    return wide

district_wide = wide_summary_table(gu_scores, radii=RADIUS_LIST)
display(district_wide.head(len(district_wide)))  # 전 구 출력


요약 Wide 저장: results/district_scores_nonhangang_WIDE_allradii.csv results/district_scores_nonhangang_WIDE_allradii.json


,구,구평균_500m_500,rank_평균_500m_500,구최고_500m_500,rank_최고_500m_500,구평균_1000m_1000,rank_평균_1000m_1000,구최고_1000m_1000,rank_최고_1000m_1000,구평균_2000m_2000,rank_평균_2000m_2000,구최고_2000m_2000,rank_최고_2000m_2000
24,중랑구,67.536050,4.0,70.1706,7.0,68.465300,1.0,77.1332,2.0,74.488600,1.0,79.6370,2.0
11,동작구,70.114600,1.0,75.6667,3.0,65.487850,2.0,71.1325,6.0,64.311300,7.0,68.6484,6.0
13,서대문구,67.151750,5.0,67.5642,9.0,65.140250,3.0,68.0754,7.0,64.004900,8.0,67.3514,8.0
15,성동구,62.877400,12.0,73.7778,4.0,64.752900,4.0,73.7778,4.0,56.148700,14.0,61.4902,12.0
9,도봉구,68.943100,2.0,71.1469,6.0,64.408900,5.0,66.6127,8.0,65.942300,3.0,71.2262,5.0
21,은평구,66.983200,6.0,67.2271,11.0,62.449000,6.0,62.6929,11.0,60.600800,11.0,60.6584,14.0
10,동대문구,63.058950,11.0,73.3958,5.0,61.529950,7.0,74.8720,3.0,61.028500,10.0,74.3772,4.0
5,광진구,68.051050,3.0,97.8820,1.0,61.102900,8.0,83.5320,1.0,66.194200,2.0,88.7418,1.0
4,관악구,65.520700,7.0,69.3459,8.0,60.986500,9.0,64.8117,9.0,63.811400,9.0,67.6366,7.0
3,강서구,62.177433,14.0,77.4773,2.0,59.149467,10.0,72.9431,5.0,64.633333,6.0,75.7680,3.0


In [688]:
# ===== Cell X: 비한강 공원 최종(반경 통합) 종합점수 & 순위 저장 =====

# 1) 반경별 총점 컬럼 목록
radius_cols = [f"총점_{r}m" for r in RADIUS_LIST if f"총점_{r}m" in non_rank.columns]

# 2) 반경 통합 종합점수: 반경별 총점의 '가중 평균'
w_series = [radii_weights.get(r, 1.0) for r in RADIUS_LIST if f"총점_{r}m" in non_rank.columns]

def weighted_mean(row, cols, weights):
    vals = [row[c] for c in cols]
    mask = ~pd.isna(vals)
    if not any(mask):
        return np.nan
    v = np.array(vals, dtype=float)[mask]
    w = np.array(weights, dtype=float)[mask]
    return float(np.average(v, weights=w))

non_overall = non_rank.copy()
non_overall["총점_overall"] = non_overall.apply(
    lambda r: weighted_mean(r, radius_cols, w_series), axis=1
)

# 3) 최종 순위(반경 통합) — NaN 허용(nullable Int64)
non_overall["rank_overall"] = non_overall["총점_overall"].rank(ascending=False, method="min").astype("Int64")

# 4) 보기 좋은 컬럼 정리
cols_show = ["공원명", "final_walk_min", "도보점수", "parking_score"] \
            + radius_cols + ["총점_overall", "rank_overall"]
cols_show = [c for c in cols_show if c in non_overall.columns]

non_overall_sorted = non_overall.sort_values("총점_overall", ascending=False)

# 5) 저장
out_csv = OUT_DIR / "parks_rank_nonhangang_OVERALL.csv"
out_json = OUT_DIR / "parks_rank_nonhangang_OVERALL.json"
non_overall_sorted[cols_show].to_csv(out_csv, index=False, encoding="utf-8-sig")
non_overall_sorted[cols_show].to_json(out_json, orient="records", force_ascii=False, indent=2)

print("✅ 비한강 공원 최종(반경 통합) 저장 완료:")
print(" -", out_csv)
print(" -", out_json)

# 6) 간단 시각화(전체 막대 그래프)
plt.figure(figsize=(10, max(6, len(non_overall_sorted) * 0.35)))
plt.barh(non_overall_sorted["공원명"], non_overall_sorted["총점_overall"])
plt.title("비한강 공원 최종(반경 통합) 종합점수 순위")
plt.xlabel("종합점수")
plt.tight_layout()
plot_path = OUT_DIR / "plot_nonhangang_overall.png"
plt.savefig(plot_path, dpi=150)
plt.close()
print(" - 그래프:", plot_path)


✅ 비한강 공원 최종(반경 통합) 저장 완료:
 - results/parks_rank_nonhangang_OVERALL.csv
 - results/parks_rank_nonhangang_OVERALL.json
 - 그래프: results/plot_nonhangang_overall.png


In [689]:
# ===== Cell X1: hangul font, weight normalization, utilities =====
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 한글 폰트(환경별로 가능한 폰트 탐색)
plt.rcParams["font.family"] = "AppleGothic" if "AppleGothic" in plt.rcParams["font.family"] or plt.rcParams.get("font.family") == [] else plt.rcParams.get("font.family")
plt.rcParams["axes.unicode_minus"] = False  # 마이너스 깨짐 방지

# 현재 가중치(원하는 비율로 자유 조정: 비율만 의미 있음)
def normalize_weights(weights: dict, available_radii):
    """가용 반경만 추려 합 1로 정규화. 전부 0 또는 유효 반경 없음이면 오류."""
    w = {r: float(weights.get(r, 0.0)) for r in available_radii}
    total = sum(w.values())
    if total <= 0:
        raise ValueError("가중치가 모두 0이거나 유효 반경이 없습니다. 적어도 하나는 0보다 커야 합니다.")
    return {r: v/total for r, v in w.items()}

def compute_weighted_overall(df: pd.DataFrame, weights: dict) -> pd.DataFrame:
    """
    반경별 총점(총점_500m/1000m/2000m)을 weights로 가중 평균 → 총점_overall 생성.
    이후 0~100 정규화 컬럼 overall_norm_0_100 추가 및 rank_overall 계산(Int64).
    """
    cols = [f"총점_{r}m" for r in weights.keys() if f"총점_{r}m" in df.columns]
    wvec = np.array([weights[r] for r in weights.keys() if f"총점_{r}m" in df.columns], dtype=float)

    def row_weighted_mean(row):
        vals = np.array([row[c] for c in cols], dtype=float)
        mask = ~np.isnan(vals)
        if not mask.any():
            return np.nan
        return float(np.average(vals[mask], weights=wvec[mask]))

    out = df.copy()
    out["총점_overall"] = out.apply(row_weighted_mean, axis=1)

    # 0~100 정규화 (Min-Max, 전부 동일/NaN이면 0 처리)
    v = out["총점_overall"].astype(float)
    vmin, vmax = np.nanmin(v.values), np.nanmax(v.values)
    if np.isfinite(vmin) and np.isfinite(vmax) and vmax > vmin:
        out["overall_norm_0_100"] = (v - vmin) / (vmax - vmin) * 100.0
    else:
        out["overall_norm_0_100"] = np.where(np.isnan(v), np.nan, 0.0)

    # 순위(내림차순, 동순위는 같은 등수) — NaN 허용(nullable)
    out["rank_overall"] = out["overall_norm_0_100"].rank(ascending=False, method="min").astype("Int64")
    return out


In [690]:
# ===== Cell X2 (수정됨): 가중치 적용 → '절대 점수'로 저장 및 시각화 =====

# 사용 가능한 반경 확인 및 가중치 정규화
available = [r for r in [500, 1000, 2000] if f"총점_{r}m" in non_rank.columns]
w_norm = normalize_weights(radii_weights, available)
print("가중치(정규화):", w_norm)

# 가중 평균 계산 (compute_weighted_overall 함수는 내부적으로 절대점수인 '총점_overall'을 이미 계산합니다)
non_overall = compute_weighted_overall(non_rank, w_norm)

# '절대 점수'인 '총점_overall'을 기준으로 정렬
non_overall_sorted = non_overall.sort_values("총점_overall", ascending=False)

# 저장 (파일명에 '_absolute'를 넣어 절대 점수 파일임을 명시)
out_csv = OUT_DIR / "parks_rank_nonhangang_OVERALL_absolute.csv"
out_json = OUT_DIR / "parks_rank_nonhangang_OVERALL_absolute.json"
non_overall_sorted.to_csv(out_csv, index=False, encoding="utf-8-sig")
non_overall_sorted.to_json(out_json, orient="records", force_ascii=False, indent=2)
print(f"✅ 절대 점수 저장 완료: {out_csv.name}, {out_json.name}")

# 시각화 (Y축 값을 절대 점수인 '총점_overall'로 변경)
plt.figure(figsize=(10, max(6, len(non_overall_sorted) * 0.35)))
plt.barh(non_overall_sorted["공원명"], non_overall_sorted["총점_overall"]) # <- 점수 컬럼 변경
# 그래프 제목과 라벨 변경
plt.title("비한강 공원 최종(반경 통합) 절대 점수 순위")
plt.xlabel("절대 점수 (가중 평균)")
plt.tight_layout()
# 그래프 파일명 변경
plot_path = OUT_DIR / "plot_nonhangang_overall_absolute.png"
plt.savefig(plot_path, dpi=150)
plt.close()
print(f"📈 그래프 저장 완료: {plot_path.name}")

# 상위 15개 미리보기 (표시할 점수 컬럼을 '총점_overall'로 변경)
display(non_overall_sorted[["공원명", "총점_overall", "rank_overall"]].head(15))

가중치(정규화): {500: 0.6, 1000: 0.3, 2000: 0.1}
✅ 절대 점수 저장 완료: parks_rank_nonhangang_OVERALL_absolute.csv, parks_rank_nonhangang_OVERALL_absolute.json
📈 그래프 저장 완료: plot_nonhangang_overall_absolute.png


,공원명,총점_overall,rank_overall
31,어린이대공원,92.66298,1
25,서울식물원,75.94611,2
34,용두근린공원(용두공원),73.93680,3
3,국립서울현충원,73.60461,4
40,중랑캠핑숲(중랑가족캠핑장),73.20602,5
24,서울숲,72.54904,6
26,서울창포원,69.79457,7
22,서울대공원,67.81471,8
11,독립공원(서대문독립공원),67.69628,9
19,불암산도시자연공원(불암산),65.26497,10


In [691]:
# ===== Cell X2a (수정본): '총점_overall' → z-score → CDF 기반 100점 환산 (0/100 방지) =====
import numpy as np
import matplotlib.pyplot as plt

base = non_overall.copy()
x = base["총점_overall"].astype(float)

# z-score
mu  = x.mean()
sig = x.std(ddof=0)
if sig == 0:
    z = np.zeros_like(x, dtype=float)
else:
    z = (x - mu) / sig

# ✅ 벡터화된 CDF 계산 (math.erf 대신 사용)
try:
    # NumPy의 벡터화된 오차함수 사용
    Phi = 0.5 * (1.0 + np.erf(z / np.sqrt(2.0)))
except AttributeError:
    # 환경에 따라 np.erf가 없으면 scipy로 대체
    from scipy.stats import norm
    Phi = norm.cdf(z)

# 안전 클리핑 (숫자 안정성)
Phi = np.clip(Phi, 1e-9, 1 - 1e-9)

# 0/100 방지용 epsilon
n   = len(base)
eps = 0.5 / n

# [0,1] -> [eps, 1-eps] -> 0~100
score = ((1 - 2*eps) * Phi + eps) * 100.0
base["zscore_cdf_100_overall"] = np.round(score, 2)

# 랭킹
zranked = base.sort_values("zscore_cdf_100_overall", ascending=False).reset_index(drop=True)
zranked["rank_overall_zcdf100"] = zranked.index + 1

# 저장
out_csv = OUT_DIR / "parks_rank_nonhangang_OVERALL_zcdf100.csv"
out_json = OUT_DIR / "parks_rank_nonhangang_OVERALL_zcdf100.json"
zranked.to_csv(out_csv, index=False, encoding="utf-8-sig")
zranked.to_json(out_json, orient="records", force_ascii=False, indent=2)
print(f"✅ CDF(0/100 방지) 100점 환산 저장 완료: {out_csv.name}, {out_json.name}")

# 시각화
plt.figure(figsize=(10, max(6, len(zranked) * 0.35)))
plt.barh(zranked["공원명"], zranked["zscore_cdf_100_overall"])
plt.title("비한강 공원 z-score(CDF) 기반 100점 환산 순위 (0/100 방지)")
plt.xlabel("z-CDF 100점 환산")
plt.tight_layout()
plot_path = OUT_DIR / "plot_nonhangang_overall_zcdf100.png"
plt.savefig(plot_path, dpi=150)
plt.close()
print(f"📈 그래프 저장 완료: {plot_path.name}")

# 상위 15개 미리보기
display(zranked[["공원명", "zscore_cdf_100_overall", "rank_overall_zcdf100", "총점_overall"]].head(15))


✅ CDF(0/100 방지) 100점 환산 저장 완료: parks_rank_nonhangang_OVERALL_zcdf100.csv, parks_rank_nonhangang_OVERALL_zcdf100.json
📈 그래프 저장 완료: plot_nonhangang_overall_zcdf100.png


,공원명,zscore_cdf_100_overall,rank_overall_zcdf100,총점_overall
0,어린이대공원,98.48,1,92.66298
1,서울식물원,90.86,2,75.94611
2,용두근린공원(용두공원),88.47,3,73.93680
3,국립서울현충원,88.03,4,73.60461
4,중랑캠핑숲(중랑가족캠핑장),87.49,5,73.20602
5,서울숲,86.55,6,72.54904
6,서울창포원,82.03,7,69.79457
7,서울대공원,78.21,8,67.81471
8,독립공원(서대문독립공원),77.96,9,67.69628
9,불암산도시자연공원(불암산),72.61,10,65.26497


In [692]:
# ===== Cell G1: 구별 점수(평균/최고) z-score→CDF 기반 0~100 환산 & 저장 =====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm


# 한글 폰트 설정(환경에 맞게 자동)
plt.rcParams["axes.unicode_minus"] = False
try:
    plt.rcParams["font.family"] = "AppleGothic"
except Exception:
    pass

def normalize_zcdf_0_100(s: pd.Series) -> pd.Series:
    """
    시리즈를 z-score → 누적정규분포(CDF)로 변환 후
    [eps, 1-eps] 구간을 0~100으로 매핑.
    - eps = 0.5 / n  (n: 유효값 개수) → 0/100이 절대 안 나오도록 완충
    - 모든 값이 같으면 50점
    - NaN은 NaN 유지
    """
    v = s.astype(float)
    mask = v.notna()
    valid = v[mask]
    n = valid.size

    out = pd.Series(np.nan, index=s.index, dtype=float)
    if n == 0:
        return out

    mu = valid.mean()
    sig = valid.std(ddof=0)

    if sig == 0:
        out[mask] = 50.0
        return out

    z = (valid - mu) / sig
    # ✅ scipy의 정규분포 CDF 사용
    Phi = norm.cdf(z)
    Phi = np.clip(Phi, 1e-12, 1 - 1e-12)

    eps = 0.5 / n
    score = ((1 - 2*eps) * Phi + eps) * 100.0

    out[mask] = np.round(score, 2)
    return out

# gu_scores: (구, 반경, 구평균_500m, 구최고_500m, rank_평균_500m, rank_최고_500m, ...) 형태
gu_norm_list = []
for r in RADIUS_LIST:
    avg_col = f"구평균_{r}m"
    top_col = f"구최고_{r}m"
    if avg_col not in gu_scores.columns or top_col not in gu_scores.columns:
        continue

    sub = gu_scores[gu_scores["반경"] == r].copy()

    # z-CDF 0~100 환산 컬럼 추가(순위는 기존 그대로 유지)
    sub[f"{avg_col}_norm100"] = normalize_zcdf_0_100(sub[avg_col])
    sub[f"{top_col}_norm100"] = normalize_zcdf_0_100(sub[top_col])

    # 보기 좋게 정렬(평균 정규화 기준 ↓)
    sub = sub.sort_values(f"{avg_col}_norm100", ascending=False, na_position="last")
    gu_norm_list.append(sub)

# 정규화 테이블 결합
gu_scores_norm = pd.concat(gu_norm_list, ignore_index=True) if gu_norm_list else gu_scores.copy()

# 저장
out_csv = OUT_DIR / "district_scores_nonhangang_norm100.csv"
out_json = OUT_DIR / "district_scores_nonhangang_norm100.json"
gu_scores_norm.to_csv(out_csv, index=False, encoding="utf-8-sig")
gu_scores_norm.to_json(out_json, orient="records", force_ascii=False, indent=2)

print("✅ 구별 정규화(z-CDF 0~100) 점수 저장:", out_csv.name, out_json.name)
display(gu_scores_norm.head(10))


✅ 구별 정규화(z-CDF 0~100) 점수 저장: district_scores_nonhangang_norm100.csv district_scores_nonhangang_norm100.json


,구,구평균_500m,구최고_500m,rank_평균_500m,rank_최고_500m,반경,구평균_1000m,구최고_1000m,rank_평균_1000m,rank_최고_1000m,구평균_2000m,구최고_2000m,rank_평균_2000m,rank_최고_2000m,구평균_500m_norm100,구최고_500m_norm100,구평균_1000m_norm100,구최고_1000m_norm100,구평균_2000m_norm100,구최고_2000m_norm100
0,동작구,70.11460,75.6667,1.0,3.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.64,82.24,NaN,NaN,NaN,NaN
1,도봉구,68.94310,71.1469,2.0,6.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.16,68.13,NaN,NaN,NaN,NaN
2,광진구,68.05105,97.8820,3.0,1.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.99,97.96,NaN,NaN,NaN,NaN
3,중랑구,67.53605,70.1706,4.0,7.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.63,64.49,NaN,NaN,NaN,NaN
4,서대문구,67.15175,67.5642,5.0,9.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.55,54.13,NaN,NaN,NaN,NaN
5,은평구,66.98320,67.2271,6.0,11.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.07,52.76,NaN,NaN,NaN,NaN
6,관악구,65.52070,69.3459,7.0,8.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.47,61.30,NaN,NaN,NaN,NaN
7,마포구,64.88660,65.2399,8.0,16.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.28,44.61,NaN,NaN,NaN,NaN
8,송파구,63.63550,65.8721,9.0,15.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.62,47.19,NaN,NaN,NaN,NaN
9,강북구,63.18660,66.7393,10.0,12.0,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.86,50.75,NaN,NaN,NaN,NaN


In [693]:
# ===== Cell G2: 구별 정규화(z-CDF 0~100) 점수 그래프(평균/최고), 반경별 저장 =====

def plot_gu_norm(df: pd.DataFrame, radius: int, which: str = "평균"):
    """
    which: '평균' 또는 '최고'
    저장 파일: plot_gu_norm_{which}_{radius}m.png
    df에는 f"구{which}_{radius}m_norm100" 컬럼이 있어야 함
    """
    base = f"구{which}_{radius}m"
    col_norm = f"{base}_norm100"
    if col_norm not in df.columns:
        print(f"[SKIP] {col_norm} 없음")
        return None

    sub = df[df["반경"] == radius].copy()
    sub = sub[["구", col_norm]].dropna()
    if sub.empty:
        print(f"[SKIP] 반경 {radius}m - 데이터 없음")
        return None

    # 내림차순 정렬 후 barh를 위해 오름차순으로 재정렬(아래→위로 커지게)
    sub = sub.sort_values(col_norm, ascending=True)

    plt.figure(figsize=(10, max(6, len(sub) * 0.35)))
    plt.barh(sub["구"], sub[col_norm])
    plt.title(f"구별 정규화 점수(z-CDF 0~100) — {which} / 반경 {radius}m")
    plt.xlabel("정규화 점수(z-CDF, 0~100)")  # 0/100 방지 스케일
    plt.tight_layout()

    outp = OUT_DIR / f"plot_gu_norm_{which}_{radius}m.png"
    plt.savefig(outp, dpi=150)
    plt.close()
    print("📈 그래프 저장:", outp)
    return outp

# 반경별로 평균/최고 그래프 모두 저장
for r in RADIUS_LIST:
    plot_gu_norm(gu_scores_norm, r, which="평균")
    plot_gu_norm(gu_scores_norm, r, which="최고")


📈 그래프 저장: results/plot_gu_norm_평균_500m.png
📈 그래프 저장: results/plot_gu_norm_최고_500m.png
📈 그래프 저장: results/plot_gu_norm_평균_1000m.png
📈 그래프 저장: results/plot_gu_norm_최고_1000m.png
📈 그래프 저장: results/plot_gu_norm_평균_2000m.png
📈 그래프 저장: results/plot_gu_norm_최고_2000m.png


In [694]:
# ===== Cell: Save final results (Hangang=park-level / Non-Hangang=district-level; Z-CDF norm) =====
import math
import numpy as np
import pandas as pd
from pathlib import Path

output_dir = Path("./final_result")
output_dir.mkdir(parents=True, exist_ok=True)

# -------------------------------
# 반경별 가중치 (논문 근거 기반 예시)
# -------------------------------
radii_weights = {500: 0.6, 1000: 0.3, 2000: 0.1}

# z-score CDF 표준화 파라미터
EPS = 0.5        # 0/100 방지 여유값 → 점수 범위 [EPS, 100-EPS] (기본 [0.5, 99.5])
Z_SHARPNESS = 1.0  # z 스케일 조절(1=기본, >1 차이 확대, <1 차이 축소)

def _normalize_zcdf_0_100_safe(series, eps=EPS, sharp=Z_SHARPNESS):
    """
    z = (x-μ)/σ 를 구하고, Φ(z)를 사용해 0~100 점수로 변환.
    - 평균은 50점, 양쪽 꼬리는 매끈하게 눌림.
    - eps로 0/100 딱 찍히는 걸 방지.
    - sharp로 분포 대비 민감도 조절(>1 더 벌어짐, <1 더 눌림).
    """
    s = pd.to_numeric(series, errors="coerce")
    v = s.dropna()
    if v.empty:
        return pd.Series(50.0, index=series.index)  # 데이터가 없으면 중간점
    mu = v.mean()
    sig = v.std(ddof=0)
    if not np.isfinite(sig) or sig == 0:
        return pd.Series(50.0, index=series.index)  # 분산 0이면 모두 같은 점수 → 50 고정

    z = ((s - mu) / sig) * float(sharp)
    # 표준정규 CDF: Φ(z) = 0.5 * (1 + erf(z / sqrt(2)))
    # np.erf는 없는 환경이 있어 math.erf 사용
    cdf = 0.5 * (1.0 + z.apply(lambda t: math.erf(t / math.sqrt(2))) )
    # 안전 마진 적용
    cdf = cdf.clip(eps / 100.0, 1.0 - eps / 100.0)
    return eps + cdf * (100.0 - 2.0 * eps)

# -------------------------------
# 공통 스코어링: (반경별 총점 → 반경가중 종합점 → Z-CDF 정규화 → 순위)
# -------------------------------
def _apply_scoring(df, walk_w, park_w, pop_w):
    df = df.copy()

    # 반경별 총점 계산 (행 단위)
    for r in radii_weights.keys():
        pop_col = f"인구점수_{r}m"
        if pop_col in df.columns:
            df[f"총점_{r}m"] = (
                df[pop_col].fillna(df[pop_col].median()) * pop_w +
                df["도보점수"].fillna(df["도보점수"].median()) * walk_w +
                df["parking_score"].fillna(df["parking_score"].median()) * park_w
            )

    # 반경 가중합 종합점
    def weighted_row(row):
        vals, ws = [], []
        for r, w in radii_weights.items():
            col = f"총점_{r}m"
            if col in row and pd.notna(row[col]):
                vals.append(row[col]); ws.append(w)
        return np.average(vals, weights=ws) if vals else np.nan

    df["종합점수"] = df.apply(weighted_row, axis=1)

    # Z-CDF 기반 0~100 표준화 (0/100 방지)
    df["종합점수_100"] = _normalize_zcdf_0_100_safe(df["종합점수"], eps=EPS, sharp=Z_SHARPNESS)

    # 순위 (내림차순)
    df["rank"] = df["종합점수_100"].rank(ascending=False, method="min").astype(int)
    return df

# -------------------------------
# 한강: 공원별 순위 산출
# -------------------------------
def compute_park_scores(parks_df, walk_w, park_w, pop_w):
    return _apply_scoring(parks_df, walk_w, park_w, pop_w)

# -------------------------------
# 비한강: 구 평균 → 순위 산출
# -------------------------------
def compute_district_scores(parks_df, walk_w, park_w, pop_w):
    df = parks_df.copy()
    # 구 정보 확보: 구_list 우선, 없으면 '구' 사용
    if "구_list" in df.columns:
        df = df.explode("구_list", ignore_index=True).rename(columns={"구_list": "구"})
    elif "구" not in df.columns:
        # 구 정보가 없으면 공원 단위라도 반환 (비어 저장 방지)
        return _apply_scoring(parks_df, walk_w, park_w, pop_w)

    df = df[df["구"].notna() & (df["구"] != "")]
    if df.empty:
        # 구가 모두 비면 공원 단위라도 반환
        return _apply_scoring(parks_df, walk_w, park_w, pop_w)

    grouped = df.groupby("구").agg({
        "도보점수": "mean",
        "parking_score": "mean",
        "인구점수_500m": "mean",
        "인구점수_1000m": "mean",
        "인구점수_2000m": "mean"
    }).reset_index()

    return _apply_scoring(grouped, walk_w, park_w, pop_w)

# -------------------------------
# 1) 한강 공원 = 공원별 순위 저장
# -------------------------------
district_han = compute_park_scores(parks_han, walk_w=1.0, park_w=1.5, pop_w=0.5)
hangang_final_path = output_dir / "hangang_final.csv"
district_han.to_csv(hangang_final_path, index=False, encoding="utf-8-sig")

# -------------------------------
# 2) 비한강 = 구 평균 순위 저장
# -------------------------------
district_non = compute_district_scores(parks_non, walk_w=0.6, park_w=0.2, pop_w=0.2)
nonhangang_final_path = output_dir / "nonhangang_final.csv"
district_non.to_csv(nonhangang_final_path, index=False, encoding="utf-8-sig")

print("저장 완료:")
print(f"- {hangang_final_path}")
print(f"- {nonhangang_final_path}")

# (선택) 상위 5 미리 확인
try:
    print("\n[한강 공원 상위 5]")
    cols = [c for c in ["공원명", "종합점수_100", "rank"] if c in district_han.columns]
    print(district_han.sort_values("rank").loc[:, cols].head(5))
except Exception:
    pass

try:
    print("\n[비한강 구 상위 5]")
    print(district_non.sort_values("rank").loc[:, ["구", "종합점수_100", "rank"]].head(5))
except Exception:
    pass


저장 완료:
- final_result/hangang_final.csv
- final_result/nonhangang_final.csv

[한강 공원 상위 5]
        공원명   종합점수_100  rank
7   여의도한강공원  91.981646     1
10   잠원한강공원  72.456453     2
5    반포한강공원  72.029062     3
6    양화한강공원  71.927291     4
3    뚝섬한강공원  63.751253     5

[비한강 구 상위 5]
       구   종합점수_100  rank
5    광진구  88.827415     1
11   동작구  88.353552     2
9    도봉구  86.383224     3
10  동대문구  82.294682     4
25   중랑구  80.804805     5
